In [17]:
import torch
import sys
from torch.utils.data import DataLoader
import torchvision.transforms as T
from tqdm.notebook import tqdm
from pathlib import Path

sys.path.append('../src/')
from newspapersdataset import (
    NewspapersDataset, 
    prepare_data_for_dataloader
)
from functions import (
    from_tsv_to_list,
    collate_fn,
    predict_eval_set,
    prepare_data_for_ap,
    calculate_map
)

sys.path.append("../../")
from image_size import get_image_size  # source: https://github.com/scardine/image_size

In [18]:
parameters = {
    'channel': 1, 
    'rescale': [1000, 1000],
    'batch_size': 16,
    'shuffle': False, 
    'num_workers': 2,
    'main_dir': f'{str(Path.cwd().parent)}',
    'gpu': False,
}

In [19]:
if torch.cuda.is_available() and parameters['gpu']:
    model = torch.load('../saved_models/model.pth')
    device = torch.device(torch.cuda.current_device())
else:
    model = torch.load('../saved_models/model.pth', map_location=torch.device('cpu'))
    device = torch.device("cpu")

In [20]:
data_transform = T.Compose([
    T.Grayscale(num_output_channels=parameters['channel']),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,)),
    ])

In [21]:
# prepare test data 
expected_test = from_tsv_to_list(parameters['main_dir']+'/preprocessed_annotations/test-A/expected.tsv')
in_file_test = from_tsv_to_list(parameters['main_dir']+'/preprocessed_annotations/test-A/in.tsv')
img_paths_test = [parameters['main_dir']+'/scraped_photos/'+path for path in in_file_test]

data_val = prepare_data_for_dataloader(
    img_dir=parameters['main_dir']+'/scraped_photos/',
    in_list=in_file_test,
    expected_list=expected_test,
    bbox_format='x0y0x1y1',
    scale=parameters['rescale'],
    test=False,
    )
dataset_val = NewspapersDataset(
    df=data_val,
    images_path=img_paths_test,
    scale=parameters['rescale'],
    transforms=data_transform,
    test=False,
    )
dataloader_val = DataLoader(
    dataset_val,
    batch_size=parameters['batch_size'],
    shuffle=parameters['shuffle'],
    collate_fn=collate_fn,
    num_workers=parameters['num_workers'],
    )

In [22]:
# make prediction on validation set
out_val, targ_val = predict_eval_set(
    dataloader=dataloader_val,
    model=model,
    device=device
)

KeyboardInterrupt: 

In [ ]:
# prepare data into a format adequate for AP calculation
prep_pred_val, prepr_gt_val = prepare_data_for_ap(out_val, targ_val)

In [ ]:
# calculate AP for each clas and mean AP based on preprocessed model results
eval_metrics_val = calculate_map(prep_pred_val, prepr_gt_val)

In [ ]:
# prepare val data 
expected_test = from_tsv_to_list(parameters['main_dir']+'/preprocessed_annotations/test-A/expected.tsv')
in_file_test = from_tsv_to_list(parameters['main_dir']+'/preprocessed_annotations/test-A/in.tsv')
img_paths_test = [parameters['main_dir']+'/scraped_photos/'+path for path in in_file_test]

data_test = prepare_data_for_dataloader(
    img_dir=parameters['main_dir']+'/scraped_photos/',
    in_list=in_file_test,
    expected_list=expected_test,
    bbox_format='x0y0x1y1',
    scale=parameters['rescale'],
    test=False,
    )
dataset_test = NewspapersDataset(
    df=data_test,
    images_path=img_paths_test,
    scale=parameters['rescale'],
    transforms=data_transform,
    test=False,
    )
dataloader_test = DataLoader(
    dataset_test,
    batch_size=parameters['batch_size'],
    shuffle=parameters['shuffle'],
    collate_fn=collate_fn,
    num_workers=parameters['num_workers'],
    )

In [ ]:
# make prediction on test set
out_test, targ_test = predict_eval_set(
    dataloader=dataloader_test,
    model=model,
    device=device
)

In [24]:
# prepare data into a format adequate for AP calculation
prep_pred_test, prepr_gt_test = prepare_data_for_ap(out_test, targ_test)

NameError: name 'out_test' is not defined

In [23]:
# calculate AP for each clas and mean AP based on preprocessed model results
eval_metrics_test = calculate_map(prep_pred_test, prepr_gt_test)

NameError: name 'prep_pred_test' is not defined